# Importing packages

In [1]:
import os
import tensorflow as tf
import cv2
import numpy as np

In [2]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Data preprocessing

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_data = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        shuffle= True)


Found 1368 images belonging to 2 classes.


# Pre-trained weights

In [36]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None,
                                classifier_activation="relu")

pre_trained_model.load_weights(r'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

for layer in pre_trained_model.layers:
    layer.trainable= True

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_564 (Conv2D)             (None, 74, 74, 32)   864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_564 (BatchN (None, 74, 74, 32)   96          conv2d_564[0][0]                 
__________________________________________________________________________________________________
activation_564 (Activation)     (None, 74, 74, 32)   0           batch_normalization_564[0][0]    
_______________________________________________________________________________________

__________________________________________________________________________________________________
activation_648 (Activation)     (None, 3, 3, 192)    0           batch_normalization_648[0][0]    
__________________________________________________________________________________________________
mixed9 (Concatenate)            (None, 3, 3, 2048)   0           activation_640[0][0]             
                                                                 mixed9_0[0][0]                   
                                                                 concatenate_12[0][0]             
                                                                 activation_648[0][0]             
__________________________________________________________________________________________________
conv2d_653 (Conv2D)             (None, 3, 3, 448)    917504      mixed9[0][0]                     
__________________________________________________________________________________________________
batch_norm

Output layer

In [43]:
last_layer= pre_trained_model.get_layer('mixed8')
print(last_layer.output_shape)
last_output= last_layer.output

(None, 3, 3, 1280)


# Building our model

In [44]:
from tensorflow.keras.optimizers import Adam

#Flatten output layer 1D
x= layers.Flatten()(last_output)

#Fully connected layer
#x= layers.Dense(1024, activation='relu')(x)
x= layers.Dense(1280, activation='relu')(x)

#Dropout due to overfitting
#x= layers.Dropout(0.2)(x)

#Sigmoid for classification
x= layers.Dense(2, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer = Adam(lr=0.001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_564 (Conv2D)             (None, 74, 74, 32)   864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_564 (BatchN (None, 74, 74, 32)   96          conv2d_564[0][0]                 
__________________________________________________________________________________________________
activation_564 (Activation)     (None, 74, 74, 32)   0           batch_normalization_564[0][0]    
____________________________________________________________________________________________

# Train model

In [45]:
model.fit(train_data, batch_size=128, epochs=10)

Epoch 1/10
43/43 [==============================] - 159s 4s/step - loss: 0.1657 - accuracy: 0.9722
Epoch 2/10
43/43 [==============================] - 153s 4s/step - loss: 0.4421 - accuracy: 0.9635
Epoch 3/10
43/43 [==============================] - 182s 4s/step - loss: 0.3500 - accuracy: 0.9759
Epoch 4/10
43/43 [==============================] - 200s 5s/step - loss: 0.2166 - accuracy: 0.9839
Epoch 5/10
43/43 [==============================] - 187s 4s/step - loss: 0.3604 - accuracy: 0.9744
Epoch 6/10
43/43 [==============================] - 176s 4s/step - loss: 0.2508 - accuracy: 0.9459
Epoch 7/10
43/43 [==============================] - 165s 4s/step - loss: 0.0558 - accuracy: 0.9868
Epoch 8/10
43/43 [==============================] - 169s 4s/step - loss: 0.0452 - accuracy: 0.9956
Epoch 9/10
43/43 [==============================] - 165s 4s/step - loss: 0.0159 - accuracy: 0.9985
Epoch 10/10
43/43 [==============================] - 177s 4s/step - loss: 0.0238 - accuracy: 0.9927


# Save model

In [72]:
checkpoint_path = r'model_save'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [73]:
import tensorflow as tf
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)
model.save_weights(checkpoint_path.format(epoch=10))

# Test data preprocessing

In [54]:
test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_data = train_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        shuffle= True)

Found 469 images belonging to 2 classes.


Test model performance

In [74]:
#reuse model saved
import tensorflow as tf
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

model.evaluate(test_data)

15/15 [==============================] - 48s 3s/step - loss: 0.1999 - accuracy: 0.9744


[0.1998840570449829, 0.9744136333465576]